In [1]:
### Evaluate the results from different configurations and compare them

#############################################
# Prerequisites - Load Libraries

In [2]:
source('MS0_Libraries.r')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on '../conda_environment/Environment_Configs.csv'”


[1] "/home/icb/corinna.losert/miniconda3/envs/jove_r//lib/R/library"


‘SeuratObject’ was built with package ‘Matrix’ 1.6.3 but the current
version is 1.6.4; it is recomended that you reinstall ‘SeuratObject’ as
the ABI for ‘Matrix’ may have changed


Attaching package: ‘SeuratObject’


The following object is masked from ‘package:base’:

    intersect


Registered S3 method overwritten by 'SeuratDisk':
  method            from  
  as.sparse.H5Group Seurat


Attaching package: ‘dplyr’


The following objects are masked from ‘package:data.table’:

    between, first, last


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::betwee

In [3]:
source('MS2_Plot_Config.r')

Warning message:
“The `size` argument of `element_line()` is deprecated as of ggplot2 3.4.0.
ℹ Please use the `linewidth` argument instead.”


In [4]:
source('MS1_Functions.r')

###############################################
# Preqrequisites Configurations & Parameters

In [5]:
### Load configurations file
global_configs = read.csv('configurations/Data_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/Data_Configs.csv'”


In [6]:
head(global_configs,2)

,parameter,value
,<chr>,<chr>
1,data_path,/lustre/groups/epigenereg01/workspace/projects/jove/input_data/
2,result_path,/lustre/groups/epigenereg01/workspace/projects/jove/results/


In [7]:
data_path = global_configs$value[global_configs$parameter == 'data_path']

In [8]:
data_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/input_data/"

In [9]:
result_path = global_configs$value[global_configs$parameter == 'result_path']

In [10]:
result_path

[1] "/lustre/groups/epigenereg01/workspace/projects/jove/results/"

In [11]:
###

In [12]:
comparison_configs = read.csv('configurations/07_Comparison_Configs.csv', sep = ',')

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
“incomplete final line found by readTableHeader on 'configurations/07_Comparison_Configs.csv'”


In [13]:
head(comparison_configs,2)

,mofa_result_name,compare_factors
,<chr>,<chr>
1,config1,"Factor1,Factor2,Factor3,Factor4,Factor5"
2,config1_weighted,"Factor1,Factor2,Factor3,Factor4,Factor5"


# Load Data 

## MOFA Models

In [14]:
model_list = list()

In [15]:
comparison_configs$mofa_result_name

[1] "config1"          "config1_weighted"

In [16]:
for(i in comparison_configs$mofa_result_name){
    model_name = paste0("03_MOFA_MODEL_", i, '.hdf5') # added number of factors in review to test different 
    outfile = file.path( paste0(result_path, '/03_results/',  model_name) )
    model_list[[as.character(i)]] <- load_model(outfile, verbose = TRUE)
    }

Loading data...

Loading expectations for 2 nodes...

Loading model options...

Loading training options and statistics...

Assigning names to the different dimensions...

Re-ordering factors by their variance explained...

Doing quality control...

Checking views names...

Checking groups names...

Checking samples names...

Checking features names...

Checking dimensions...

Checking there are no features with complete missing values...

Checking sample covariates...

Checking expectations...

Checking for intercept factors...

Warning message in .quality_control(object, verbose = verbose):
“Factor(s) 5, 9, 10, 20 are strongly correlated with the total number of expressed features for at least one of your omics. Such factors appear when there are differences in the total 'levels' between your samples, *sometimes* because of poor normalisation in the preprocessing steps.
”
Checking for highly correlated factors...

Loading data...

Loading expectations for 2 nodes...

Loading model op

In [ ]:
names(model_list)

In [ ]:
#str(model_list)

In [ ]:
print(length(model_list))

# Downstream Analysis of generated models

## Compare total explained variance

In [20]:
lapply(model_list, function(x){
    mean(x@cache$variance_explained$r2_total[[1]])
    })
### 20 Factor model used: 35.7461964551804   

$config1
[1] 36.53926

$config1_weighted
[1] 36.78307

In [21]:
### compare explained variance per view

In [22]:
var_explained_per_view = lapply(model_list, function(x){
    x@cache[["variance_explained"]]$r2_total
    })

In [23]:
var_explained_per_view_data = data.frame()

for(i in names(var_explained_per_view)){
    data = var_explained_per_view[[i]]$group1
    data = as.data.frame(data)
    data$view = rownames(data)
    data$view = str_replace_all(data$view, ' ', '')
    data$view = str_replace_all(data$view, '\\.', '')
    data$configuration = i
    
    var_explained_per_view_data = rbind(data, var_explained_per_view_data)
    }

In [24]:
head(var_explained_per_view_data,2)

,data,view,configuration
,<dbl>,<chr>,<chr>
B.cell,31.04474,Bcell,config1_weighted
CD14.Mono,49.80700,CD14Mono,config1_weighted


In [25]:
unique(var_explained_per_view_data$configuration)

[1] "config1_weighted" "config1"

In [26]:
figure_name = paste0("FIG07_Variance_Model_Comparison")

In [27]:
# Sizes of the plot
width_par = 6
height_par =4

In [28]:
plot = ggplot(var_explained_per_view_data, aes(x=view, y=data, fill=configuration, order = configuration)) + 
    geom_bar(position="dodge", stat="identity") + ylab('amount_explained_variance') + plot_config + scale_fill_okabe_ito() + coord_flip() + ylab('Total percentage of explained variance')
  # TBD: improve coloring + selection

#plot
pdf(paste0('figures/07_figures/', figure_name, '.pdf'), width =width_par, height =height_par)
print( plot)
dev.off()

png 
  2

## Compare factor weights of samples

In [29]:
factor_weights = lapply(model_list, function(x){
    get_factors(x, factors = 'all')$group1
    })

In [30]:
factor_weight_data = data.frame()

for(i in names(factor_weights)){
    data = factor_weights[[i]]
    data = as.data.frame(data)
    data$configuration = i
    data$sample = rownames(data)
    data = melt(data)
    ## filter on relevant factors
    data = data[data$variable %in% unlist(str_split(comparison_configs$compare_factors[comparison_configs$mofa_result_name == i], ',')),]
    
    
    factor_weight_data = rbind(data, factor_weight_data)
    }

Using configuration, sample as id variables

Using configuration, sample as id variables



In [31]:
factor_weight_data$configuration = paste0(factor_weight_data$variable, '_', factor_weight_data$configuration)

In [32]:
head(factor_weight_data,2)

,configuration,sample,variable,value
,<chr>,<chr>,<fct>,<dbl>
1,Factor1_config1_weighted,k1,Factor1,-3.173985
2,Factor1_config1_weighted,k10,Factor1,-1.237786


In [33]:
unique(factor_weight_data$variable)

[1] Factor1 Factor2 Factor3 Factor4 Factor5
20 Levels: Factor1 Factor2 Factor3 Factor4 Factor5 Factor6 Factor7 ... Factor20

In [34]:
cor_data = factor_weight_data %>% dcast(sample ~ configuration, value.var = 'value')

In [35]:
head(cor_data,2)

,sample,Factor1_config1,Factor1_config1_weighted,Factor2_config1,Factor2_config1_weighted,Factor3_config1,Factor3_config1_weighted,Factor4_config1,Factor4_config1_weighted,Factor5_config1,Factor5_config1_weighted
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,k1,0.6644081,-3.173985,-1.9516023,0.4933851,1.1575432,0.4103477,-0.1543102,0.1916941,0.9254733,0.5197276
2,k10,0.8733989,-1.237786,-0.8954186,0.9258907,0.4146406,0.2647406,0.7414734,0.7059813,-0.2085353,-0.9842337


In [36]:
cor_data$sample = NULL
correlation = cor(cor_data, use = 'pairwise.complete.obs')

In [37]:
head(correlation,2)

,Factor1_config1,Factor1_config1_weighted,Factor2_config1,Factor2_config1_weighted,Factor3_config1,Factor3_config1_weighted,Factor4_config1,Factor4_config1_weighted,Factor5_config1,Factor5_config1_weighted
Factor1_config1,1.00000000,-0.07051291,-0.03112852,0.97593446,-0.01624718,-0.064847811,-0.007363368,-0.002623532,0.05104008,0.07876028
Factor1_config1_weighted,-0.07051291,1.00000000,0.79824778,-0.06773175,-0.09264334,0.001222464,-0.151984353,-0.110519205,0.05975563,0.08554158


In [38]:
figure_name = paste0("FIG07_Factor_Correlations")

In [39]:
# Sizes of the plot
width_par = 8
height_par =5

In [40]:
options(repr.plot.width=40, repr.plot.height=20)
pdf(paste0('figures/07_figures/', figure_name, '.pdf'), width =width_par, height =height_par)
corrplot(correlation, method = 'number',  type = 'upper', col = COL2('BrBG'), tl.col = 'black', tl.cex = 0.6,   number.cex = 0.4, tl.offset = 0.2, diag = FALSE) + plot_config + theme(text = element_text(size = 5))
dev.off()

NULL

png 
  2

In [41]:
## Generate Table

In [42]:
correlation = melt(correlation)

In [43]:
correlation = correlation[correlation$Var1 != correlation$Var2,]  # remove diagonal entries

In [44]:
colnames(correlation) = c('Factor_Config1', 'Factor_Config2', 'Pearson_Correlation')

In [45]:
head(correlation ,2)

,Factor_Config1,Factor_Config2,Pearson_Correlation
,<fct>,<fct>,<dbl>
2,Factor1_config1_weighted,Factor1_config1,-0.07051291
3,Factor2_config1,Factor1_config1,-0.03112852


In [46]:
## Save result
write.csv(correlation, paste0(result_path, '/07_results/07_Factor_Correlations',  '.csv'), row.names = FALSE)

## Compare feature weights

In [47]:
weights = lapply(model_list, function(x){
    get_weights(x, views = "all", factors = "all")
    })

In [48]:
weight_data = data.frame()

In [49]:
for(j in names(weights)){
    weights_factor_amount = weights[[j]]

    for (i in names(weights_factor_amount)){
        data = data.frame(weights_factor_amount[[i]])
        data$view = i
        data$configuration = j
        data$feature = rownames(data)
        
        data = melt(data)
        # filter on relevant factors for comparison
        data = data[data$variable %in% unlist(str_split(comparison_configs$compare_factors[comparison_configs$mofa_result_name == j], ',')),]
        
        weight_data = rbind(weight_data,data)
        }
    }

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, configuration, feature as id variables

Using view, 

In [50]:
head(weight_data,2)

,view,configuration,feature,variable,value
,<chr>,<chr>,<chr>,<fct>,<dbl>
1,B.cell,config1,B.cell__ACTB,Factor1,-0.21732116
2,B.cell,config1,B.cell__ACTG1,Factor1,-0.03822634


In [51]:
weight_data$configuration = paste0(weight_data$variable, '_',weight_data$configuration)
weight_data$view_feature = paste0(weight_data$view, '__', weight_data$feature)
weight_data$view_feature =  str_replace_all(weight_data$view_feature, ' ', '')
weight_data$view_feature = str_replace_all(weight_data$view_feature, '\\.', '')

In [52]:
head(weight_data,2)

,view,configuration,feature,variable,value,view_feature
,<chr>,<chr>,<chr>,<fct>,<dbl>,<chr>
1,B.cell,Factor1_config1,B.cell__ACTB,Factor1,-0.21732116,Bcell__Bcell__ACTB
2,B.cell,Factor1_config1,B.cell__ACTG1,Factor1,-0.03822634,Bcell__Bcell__ACTG1


In [53]:
cor_data = weight_data %>% dcast(view_feature ~ configuration, value.var = 'value')

In [54]:
head(cor_data,2)

,view_feature,Factor1_config1,Factor1_config1_weighted,Factor2_config1,Factor2_config1_weighted,Factor3_config1,Factor3_config1_weighted,Factor4_config1,Factor4_config1_weighted,Factor5_config1,Factor5_config1_weighted
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Bcell__Bcell__ACTB,-0.21732116,0.1839543,0.18824680,-0.18950829,-0.11174294,0.002446882,0.08027010,-0.006809727,-0.06805549,-0.038558220
2,Bcell__Bcell__ACTG1,-0.03822634,0.0346680,-0.02855239,-0.03312262,-0.04507595,-0.007437024,0.06292246,-0.014656595,-0.05201821,0.001974613


In [55]:
cor_data$view_feature = NULL
correlation = cor(cor_data, use = 'pairwise.complete.obs')

In [56]:
head(correlation,2)

,Factor1_config1,Factor1_config1_weighted,Factor2_config1,Factor2_config1_weighted,Factor3_config1,Factor3_config1_weighted,Factor4_config1,Factor4_config1_weighted,Factor5_config1,Factor5_config1_weighted
Factor1_config1,1.00000000,0.02126009,-0.01879164,0.9827013,-0.007151819,-0.0229192,-0.06781573,-0.1259136,0.021504795,0.03562152
Factor1_config1_weighted,0.02126009,1.00000000,0.93055251,0.1095240,-0.021922091,0.1138339,-0.08849836,0.1422697,0.008213468,-0.14706355


In [57]:
figure_name = paste0("FIG07_Feature_Correlations")

In [58]:
# Sizes of the plot
width_par = 8
height_par =5

In [59]:
options(repr.plot.width=40, repr.plot.height=20)
pdf(paste0('figures/07_figures/', figure_name, '.pdf'), width =width_par, height =height_par)
corrplot(correlation, method = 'number',  type = 'lower', col = COL2('BrBG'), tl.col = 'black', tl.cex = 0.6, tl.srt = 90, number.cex = 0.4, tl.offset = 0.2, diag = FALSE) + plot_config + theme(text = element_text(size = 5))
dev.off()

NULL

png 
  2

In [60]:
## Generate Table

In [61]:
correlation = melt(correlation)

In [62]:
correlation = correlation[correlation$Var1 != correlation$Var2,]  # remove diagonal entries

In [63]:
colnames(correlation) = c('Factor_Config1', 'Factor_Config2', 'Pearson_Correlation')

In [64]:
head(correlation ,2)

,Factor_Config1,Factor_Config2,Pearson_Correlation
,<fct>,<fct>,<dbl>
2,Factor1_config1_weighted,Factor1_config1,0.02126009
3,Factor2_config1,Factor1_config1,-0.01879164


In [65]:
## Save result
write.csv(correlation, paste0(result_path, '/07_results/07_Feature_Correlations',  '.csv'), row.names = FALSE)